In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
%matplotlib inline

In [3]:
path = "datasets/"

In [4]:
!ls datasets/

item_categories.csv		items-translated.csv  test.csv
item_categories-translated.csv	sales_train.csv
items_russian.csv		shops-translated.csv


In [5]:
shops = pd.read_csv(path + "shops-translated.csv")
items = pd.read_csv(path + "items-translated.csv")
item_categories = pd.read_csv(path + "item_categories-translated.csv")
items_russian = pd.read_csv(path + "items_russian.csv")
train = pd.read_csv(path + "sales_train.csv")
test = pd.read_csv(path + "test.csv")

In [6]:
item_categories[:10]

,"item_category_id,item_category_name_translated"
0,"0,PC - Headsets / Headphones"
1,"1,Accessories - PS2"
2,"2,Accessories - PS3"
3,"3,Accessories - PS4"
4,"4,Accessories - PSP"
5,"5,Accessories - PSVita"
6,"6,Accessories - XBOX 360"
7,"7,Accessories - XBOX ONE"
8,"8,Tickets (figure)"
9,"9,Delivery of goods"


In [7]:
item_categories = item_categories["item_category_id,item_category_name_translated"].str.split(',', expand=True)

In [8]:
item_categories.columns = ["item_category_id", "item_category_name", "c", "d"]

In [9]:
item_categories = item_categories.drop(["c", "d"], axis=1)

In [10]:
item_categories[:10]

,item_category_id,item_category_name
0,0,PC - Headsets / Headphones
1,1,Accessories - PS2
2,2,Accessories - PS3
3,3,Accessories - PS4
4,4,Accessories - PSP
5,5,Accessories - PSVita
6,6,Accessories - XBOX 360
7,7,Accessories - XBOX ONE
8,8,Tickets (figure)
9,9,Delivery of goods


In [11]:
item_categories.isnull().sum()/len(item_categories)

item_category_id      0.0
item_category_name    0.0
dtype: float64

In [12]:
item_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_id    84 non-null     object
 1   item_category_name  84 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


In [13]:
shops[:10]

,"shop_id,shop_name_translated"
0,"0,""Yakutsk Ordzhonikidze, 56 francs"""
1,"1,""Yakutsk TC """"Central"""" franc"""
2,"2,""Adygea Shopping Center """"Mega"""""""
3,"3,""Balashikha TRK """"October-Kinomir"""""""
4,"4,""Volzhsky shopping center """"Volga Mall"""""""
5,"5,""Vologda Shopping and Entertainment Center ""..."
6,"6,""Voronezh (Plekhanovskaya, 13)"""
7,"7,""Voronezh TRC """"Maksimir"""""""
8,"8,""Voronezh TRC City-Park """"Grad"""""""
9,"9,Outbound Trade"


In [ ]:
shops = shops["shop_id,shop_name_translated"].str.split(',', expand=True)

In [ ]:
shops.columns = ["shop_id", "shop_name", "a"]

In [ ]:
shops = shops.drop("a", axis=1)

In [ ]:
shops[:10]

In [ ]:
shops.isnull().sum()/len(shops)

In [ ]:
shops.info()

In [ ]:
shops["shop_id"] = shops["shop_id"].astype(int)

In [ ]:
items[:10]

In [ ]:
items.columns = ["item_id", "item_name"]

In [ ]:
items

In [ ]:
items.isnull().sum()/len(items)

In [ ]:
train

In [ ]:
T1 = pd.merge(train, items, on="item_id", how='right')

In [ ]:
T1.isnull().sum()

In [ ]:
T1["shop_id"] = T1["shop_id"].fillna(0)

In [ ]:
T1["shop_id"].astype(int)

In [ ]:
train = pd.merge(T1, shops, on='shop_id', how='outer')

In [ ]:
train

In [ ]:
item_categories["item_category_id"] = item_categories["item_category_id"].astype(float)

In [ ]:
items_russian["item_category_id"] = items_russian["item_category_id"].astype(float)

In [ ]:
T1 = pd.merge(train, items_russian, on='item_id', how='outer')

In [ ]:
T2 = pd.merge(T1, item_categories, on='item_category_id', how='outer')

In [ ]:
train = T2.drop("item_name_y", axis=1)

In [ ]:
train.columns = ["date", "date_block", "shop_id", "item_id", "item_price", "item_count_day", "item_name", "shop_name", "item_category_id", "item_category_name"]

In [ ]:
train

In [ ]:
train = train.reindex(columns= ["date_block", "date", "shop_id", "shop_name", "item_category_id", "item_category_name", "item_id", "item_name", "item_price", "item_count_day"])

In [ ]:
train

In [ ]:
train.isnull().sum()

In [ ]:
null_rows = train[train.isna().any(axis=1)]
null_rows

In [ ]:
len(null_rows.index)

In [ ]:
train = train.dropna()

In [ ]:
train

In [ ]:
train.dtypes

In [ ]:
train['date'] = pd.to_datetime(train['date'])

In [ ]:
train

## Plots

In [ ]:
plt.figure(figsize=(10,5))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price)

plt.figure(figsize=(10,5))
plt.xlim(train.item_count_day.min(), train.item_count_day.max()*1.1)
sns.boxplot(x=train.item_count_day)

In [ ]:
train = train[train.item_price<100000]
train = train[train.item_count_day<2000]

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(train['item_count_day'], train['item_price'])
plt.xlabel("item count per day")
plt.ylabel("item price")

#### Groupby shop:

In [ ]:
shop_details = train.groupby(["shop_id", "date_block", "date", "item_category_id", "item_id"]).agg(np.mean)

In [ ]:
shop_details

### Single shop details:

In [ ]:
shop_20 = train[(train['shop_id'] == 20.0)]

In [ ]:
shop_20 = shop_20.drop("shop_id", axis=1)

In [ ]:
shop_20

In [ ]:
plt.figure(figsize = (12, 6))
sns.stripplot(x=shop_20["item_price"], color='red')

plt.figure(figsize = (12, 6))
sns.stripplot(x=shop_20["item_count_day"], color='blue')

In [ ]:
plt.figure(figsize = (20,20))
plt.bar(shop_20["item_category_name"], shop_20["item_count_day"])
plt.title("item categories vs count sold per day")
plt.xticks(rotation=90)

In [ ]:
plt.scatter(shop_20["item_count_day"], shop_20["item_price"])

In [ ]:
plt.scatter(shop_20["date"], shop_20["item_count_day"])
plt.xticks(rotation=90)

In [ ]:
shop_20_item_categories = shop_20[["item_category_name", "item_category_id"]]
shop_20_item_categories = shop_20_item_categories.drop_duplicates() 
shop_20_item_categories = shop_20_item_categories.sort_values(by="item_category_id")
shop_20_item_categories

In [ ]:
%matplotlib qt

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
xs=shop_20["item_category_id"]
zs=shop_20["item_price"]
ys=shop_20["item_count_day"]
shop_20_plot = ax.scatter(xs, ys, zs, c=xs)
ax.set_xlabel("item category")
ax.set_ylabel("item count")
ax.set_zlabel("item price")
plt.title("Shop 20 - Items by category: \n number sold and price")

purple_patch = mpatches.Patch(color='purple', label='Books - Artbook')
darkpurple_patch = mpatches.Patch(color='rebeccapurple', label='Books - Comics')
teal_patch = mpatches.Patch(color='teal', label='Gifts - Attributes')
seagreen_patch = mpatches.Patch(color='lightseagreen', label='Gifts - Soft Toys')
aquamarine_patch = mpatches.Patch(color='mediumaquamarine', label='Gifts - Board Games')
lightgreen_patch = mpatches.Patch(color='lightgreen', label='Gifts - Souvenirs')
yellow_patch = mpatches.Patch(color='yellow', label='Elements of a food')
plt.legend(bbox_to_anchor=(0, 0, 0, 1), handles=[purple_patch, darkpurple_patch, teal_patch, seagreen_patch, aquamarine_patch, lightgreen_patch, yellow_patch])

### Single product:

In [ ]:
train

##### by item:

In [ ]:
plt.scatter(train["item_id"], train["item_price"], s=50,
             alpha=0.5,)

In [ ]:
random_selection = train.sample(n=1)
random_selection

###### choose this random selections "item_id" and do analysis on this item throughout time in all shops

In [ ]:
random_item = train[(train['item_id'] == 7912)]
random_item = random_item.drop(["item_id", "item_name", "item_category_id", "item_category_name"], axis=1)
random_item

In [ ]:
h = random_item.groupby("date")["item_count_day"].count()
h

In [ ]:
%matplotlib inline

In [ ]:
plt.scatter(random_item["date"], random_item["item_price"], s=50,
             alpha=0.5,)